In [1]:
# import modules
import config
import tempfile, os
import gpml
import experiment as exp

In [4]:
# plot results
MATLAB_PLOT_FULL_POSTERIOR = r'''
load '%(datafile)s'
load '%(params_file)s'

addpath(genpath('%(gpml_path)s'));
addpath(genpath('%(matlab_script_path)s'));

X = double(X);
y = double(y);
M = size(y,2);
covfunc_fS = %(kernel_syntax)s;
covfunc_f = {@covSMfast, 10};
hyp_fS = hyp_opt.cov;

num_interpolation_points = %(interp_points)s;

left_extend = 0.0;
right_extend = 0.05;

x_left = min(X) - (max(X) - min(X))*left_extend;
x_right = max(X) + (max(X) - min(X))*right_extend;
xrange = linspace(x_left, x_right, num_interpolation_points)';
xrange_no_extrap = linspace(min(X), max(X), num_interpolation_points)';

figname = '%(figname)s';

for i=1:M
    hyp.cov = [scale{i};hyp_fS;hyp_f{i}];
    covfunc = {@covSum, {{@covSum, {{@covConst}, {@covProd, {@covConst, covfunc_fS}}}}, covfunc_f}};
    meanfunc = {@meanZero};
    likfunc = {@likDelta};

    complete_sigma = feval(covfunc{:}, hyp.cov, X);
    complete_sigmastar = feval(covfunc{:}, hyp.cov, X, xrange);
    complete_sigmastarstart = feval(covfunc{:}, hyp.cov, xrange);
    
    complete_mean = complete_sigmastar' / complete_sigma * y(:, i);
    complete_var = diag(complete_sigmastarstart - complete_sigmastar' / complete_sigma * complete_sigmastar);

    figure(i+1); clf; hold on;
    mean_var_plot( X, y(:,i), ...
               xrange, complete_mean, ...
               2.*sqrt(complete_var), false, false);

    title(sprintf('Full model posterior with extrapolations - %%s', S(i,:)));
    filename = sprintf('%%s_all-%%d.png', figname, i);
    saveas( gcf, filename );
end

'''

def plot_full_posterior(data_folder, results_dir, save_folder, num_interpolation_points=2000):
    
    if not isinstance(results_dir, list):
        results_dir = [results_dir]

    data_sets = list(exp.gen_all_datasets(data_folder))
    for r, file in data_sets:
        results_files = []
        for folder in results_dir:
            results_file = os.path.join(folder, file + "_result.txt")
            if os.path.isfile(results_file):
                results_files.append(results_file)
        if len(results_files) > 0:
            datafile = os.path.join(r, file + ".mat")
            data = gpml.my_load_mat(datafile)
            X = data[0]
            y = data[1]
            M = y.shape[1]
            names = data[5]
        shared_model, depth = exp.parse_results(results_files)
        fig_folder = os.path.join(save_folder, 'posteriror')
        if not os.path.exists(fig_folder):
            os.makedirs(fig_folder)
    
        (fd1, temp_data_file) = gpml.my_standardise_and_save_data(X, y, names)
        # insert your path
        matlab_dir = '/home/smheo/open-software/Relational-Automatic-Statistician/source/matlab'
        figname=os.path.join(fig_folder, file)       
        # hand out matlab code
        code = MATLAB_PLOT_FULL_POSTERIOR
        code = code % {'datafile': temp_data_file,
            'gpml_path':config.GPML_PATH,
            'matlab_script_path': matlab_dir,
            'kernel_syntax': shared_model.kernel.get_gpml_expression(dimensions=X.shape[1]),
            'params_file': results_files[0]+'lvl_'+str(depth)+'_0.mat1.mat',
            'figname': figname,
            'interp_points': str(num_interpolation_points)
            }
        gpml.run_matlab_code(code, verbose=True, jvm=True)
     
        os.close(fd1)
        os.remove(temp_data_file)


In [6]:
# plot
plot_full_posterior("../srkl-data/stocks", "../srkl-results/", "../srkl-report/", 2000)

['/usr/local/MATLAB/R2015b/bin/matlab', '-nosplash', '', '-nodisplay']

Script file (/tmp/tmpVf54Q4.m) contents : ==========================================

load '/tmp/tmpmWiinL.mat'
load '../srkl-results/stocks-0_result.txtlvl_4_0.mat1.mat'

addpath(genpath('/home/smheo/open-software/Relational-Automatic-Statistician/source/gpml'));
addpath(genpath('/home/smheo/open-software/Relational-Automatic-Statistician/source/matlab'));

X = double(X);
y = double(y);
M = size(y,2);
covfunc_fS = {@covSum, {{@covChangeWindowMultiD, {1, {@covProd, {{@covSEiso}, {@covSum, {{@covChangeWindowMultiD, {1, {@covSEiso}, {@covSEiso}}}, {@covConst}}}}}, {@covSEiso}}}, {@covNoise}}};

covfunc_f = {@covSMfast, 10};

hyp_fS = hyp_opt.cov;

num_interpolation_points = 2000;

left_extend = 0.0;
right_extend = 0.05;

x_left = min(X) - (max(X) - min(X))*left_extend;
x_right = max(X) + (max(X) - min(X))*right_extend;
xrange = linspace(x_left, x_right, num_interpolation_points)';
xrange_no_extrap = linspace(min(X), 

![fg](../srkl-report/posteriror/stocks-0_all-1.png)

![fg](../srkl-report/posteriror/stocks-0_all-2.png)

![fg](../srkl-report/posteriror/stocks-0_all-3.png)